In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


##  Useful Functions

1) display_img : this function is to display the image by passing the image array (For quick visualization purpose)

2) mmps_to_mph : this function is to convert the speed from mm/s to mile/hour(mph) as mph is more commonly used in baseball tournament 

3) display_img_vector : this function is to display image with the baseball detected + speed vector annotations and text annotations after the speeds are being computed

In [ ]:
def display_img(img,label):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')
    ax.axis('off')
    ax.set_title(label)


In [ ]:
def mmps_to_mph(speed):
    return speed * 0.00223694

In [ ]:
def display_img_vector(img,label,vector,speed,count):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img)

    # Draw velocity vector
    ax.arrow(vector[0],vector[1],-100,0,width=2,color='r')
    ax.arrow(vector[0],vector[1],0,-100,width=2,color='y')
    ax.scatter(vector[0], vector[1], s=30, c='g', marker='X')

    # Text annotation with speed computed corresponding velocity veector 
    ax.text(vector[0]-100,vector[1]+50,str(speed[0])+'mph',fontsize=10,color='r',bbox={'facecolor': 'black', 'alpha': 0.75, 'pad': 5})
    ax.text(vector[0]+40,vector[1],str(speed[1])+'mph',fontsize=10,rotation=90,color='y',bbox={'facecolor': 'black', 'alpha': 0.75, 'pad': 5})
    ax.text(vector[0]-150,vector[1]-50,str(speed[2])+'mph',fontsize=10,color='g',bbox={'facecolor': 'black', 'alpha': 0.75, 'pad': 5})

    # Text annotation explanation corresponding to the colored annotation in the image
    ax.text(1024,1100,'Average Speed Vector=['+str(speed[0])+'mph'+' , '+str(speed[1])+'mph'+ ' , '+str(speed[2])+'mph'+']',fontsize=12,color='white',horizontalalignment='right')
    ax.text(1024,1150,'Red Arrow: x-direction',fontsize=12,color='r',horizontalalignment='right')
    ax.text(1024,1200,'Yellow Arrow: y-direction',fontsize=12,color='y',horizontalalignment='right')
    ax.text(1024,1250,'Green Cross: z-direction (Normal to the image plane)',fontsize=12,color='g',horizontalalignment='right')
    
    ax.axis('off')
    ax.set_title(label)
    fig.savefig('img_final/IMG_FINAL'+str(count)+'.png',bbox_inches='tight')

# Start

### 1st Part
- Mannually find the region of interest for the baseball relative to each image. Record the pixel position and save the ROI as image templates in 'img_template' folder

### 2nd Part
- Use the openCV 'matchTample' function to detect the ball position relative to each template being saved
- Then, use the openCV 'HoughCircles' function to detect detect the edges of the baseball so that the radius(pixel) and the center of position (pixel) for the baseball can be obtained

### 3rd Part
- Use Triangle Similarity to calculate necessary parameters to compute velocity in x(width),y(height),z(normal to the image plane) direction
- Calculate velocity in x,y,z direction
- Compute image with detected baseball + velocity vector and save the images into 'img_final' folder

In [ ]:
######### 1st Part #########

# Baseball ROI 
top_left  = [[770,530],[780,530],[780,520],[780,490],[770,460],[760,425],[750,390],[750,350],[740,320],[730,270],[720,230],[705,175],[690,120],[680,60],[670,0]]
bottom_right = [[810,570],[820,570],[820,560],[820,530],[810,500],[810,475],[800,440],[800,400],[790,370],[780,320],[770,280],[760,230],[750,180],[740,120],[730,60]]

count = 1


# Crop the ROI and save the image in to 'img_template' folder
for a,b in zip(top_left,bottom_right):
    h1,w1 = a[0],a[1]
    h2,w2 = b[0],b[1]
    img_temp = cv2.imread('img/IMG'+str(count)+'.bmp')
    plt.imsave('img_template/template'+str(count)+'.bmp',img_temp[h1:h2,w1:w2])
    count += 1



In [ ]:
######### 2nd Part #########


##** This whole cell is about template matching **##

# Create some list to record down some computed results 
img_list =[] # IMG data
img_rect_labeled_list = [] # IMG data with the labelled rectangle for baseball
# Rectangle position (top left and bottom right pixel) for the baseball ROI relative to each image
top_left_list = [] 
bottom_right_list = []

# Method to use for 'matchTemplate' function 
methods = 'cv2.TM_CCOEFF_NORMED'
methods = eval(methods)

# Initialize subplots
fig,axes = plt.subplots(3,5,figsize=(20,15),dpi=200)
axes = axes.ravel()
count = 1

for ax in axes:

    # Read and load IMG
    img_bb = cv2.imread('img/IMG'+str(count)+'.bmp')
    img_copy = img_bb.copy()
    img_bb_gray = cv2.cvtColor(img_bb, cv2.COLOR_BGR2GRAY)
    template_bb = cv2.imread('img_template/template'+str(count)+'.bmp',0)
    img_list.append(img_copy)
    h,w = template_bb.shape


    # matchTemplate function
    res = cv2.matchTemplate(img_bb_gray,template_bb,methods)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    img_rect_labeled_list.append(img_bb)
    top_left_list.append(top_left)
    bottom_right_list.append(bottom_right)

    # Draw the ROI rectangle for baseball position relative to each image
    cv2.rectangle(img_bb,top_left, bottom_right, (255,0,0), 2)
    ax.imshow(img_bb)
    ax.axis('off')
    ax.set_title('IMG'+str(count))
    count +=1

In [ ]:
# Display an image with labelled rectangle in case the top picture is too small for you to see
display_img(img_rect_labeled_list[4],'IMG5')

In [ ]:
##** This whole cell is about circle detection within the ROI rectangle computed above **##

# Initialize subplots
fig,axes = plt.subplots(3,5,figsize=(20,15),dpi=200)
axes = axes.ravel()
count = 1

# Create some list to record down some computed results 
circle_list = []
img_circle_labeled_list = []

for ax in axes:

    # Load image
    img_bb = img_list[count-1]
    img_bb_copy = img_bb.copy()
    img_bb_gray = cv2.cvtColor(img_bb_copy,cv2.COLOR_BGR2GRAY)
    img_bb_gray = cv2.blur(img_bb_gray,(3,3))


    # Load ROI (pixel) from the list computed on top
    top_left = top_left_list[count-1]
    bottom_right = bottom_right_list[count-1]

    # HoughCircles function to detect the circle within the ROI
    circles = cv2.HoughCircles(img_bb_gray[top_left[1]:bottom_right[1],top_left[0]:bottom_right[0]], cv2.HOUGH_GRADIENT,1, 100, param1=110, param2=15, minRadius=0, maxRadius=0)
    circles = np.uint16(np.around(circles))
    a,b,r = circles[0,0,0],circles[0,0,1],circles[0,0,2]
    a,b,r = a+top_left[0],b + top_left[1],r
 
    # Append the circle position(pixel) to the list
    circle_list.append([a,b,r])

    # Draw the detected circle on the full image
    cv2.circle(img_bb_copy,(a,b),r,(0,0,255),2)
    img_circle_labeled_list.append(img_bb_copy)
    ax.imshow(img_bb_copy)
    ax.axis('off')
    ax.set_title('IMG'+str(count))
    count +=1

In [ ]:
# Display an image with labelled circle in case the top picture is too small for you to see
display_img(img_circle_labeled_list[4],'IMG5')

In [ ]:
######### 3rd Part #########


# Initialize camera parameter
FL = 8 # mm
res_w = 1024 # pixel
res_h = 1280 # pixel
pixel_size = 0.0048 # mm per pixel
dist = 4*1000 # mm
tilt_angle = 10*np.pi/180 # rad
rad_bb = 37.3 # mm
fps = 240 # frame per second

# Calculate true working distance with 10 degree tilt angle
wd = dist/np.cos(tilt_angle) # mm

# Calculate diameter of object
diameter_bb = rad_bb*2 # mm


# Calculate sensor width and height in mm
sensor_w = res_w * pixel_size # mm
sensor_h = res_h * pixel_size # mm

# Calculate focal length (pixel) relative to width and heigh of the sensor (Triangle Similarity method)
fx = FL*res_w/sensor_w # pixel
fy = FL*res_h/sensor_h # pixel

# Calculate angle of view and field of view for this specific camera (Triangle Similarity method)
AoV_H = 2*np.arctan(sensor_h/(2*FL)) # rad
AoV_W = 2*np.arctan(sensor_w/(2*FL)) # rad
FoV_H = 2* wd* np.tan(AoV_H/2) # mm
FoV_W = 2* wd* np.tan(AoV_W/2) # mm


# Approaches to calculate velocity vector 

- I take IMG 5,8,10,11,12,13,14,15 since these pictures are detected with perfect labelled circle using HoughCircles function as shown above

- I do not take IMG 1 and 2 because the ball was not being hit off from the bat in these 2 IMG

- I do not take IMG 3,4,6,7,9 because the labelled circle detected from HoughCircles function was not perfectly fit to the balls. That means the radius and center of position for these circles are not accurate

- In real life, it is not practical to tell people the speed of the baseball in each frame captured under 240FPS camera. Instead, I calculate all the speed for the chosen IMG and take the average of them. Eventually, we will call it as the average speed of the baseball when it was being hit off from the bat

- As we know, speed = distance_travelled/time_travelled. {**v = dx/dt**}

- Using the Triangular Similarity method, I can compute position(relative to the center of position for the baseball at that particular frame) in millimeter(mm) for x and y direction. Then, I can use the equation: {**v = dx/dt = abs(x1-x2)/(t1-t2)**} ,to calculate the velocity in that particular axis

- Similar to z direction (normal to image plane), I calculate the depth distance between the camera and the baseball using the equation: {**Z = FL_P*D/P**} ,where Z = depth distance between camera and ball in millimeter(mm), FL_P = focal length in pixel, D = baseball diameter in millimeter(mm), P = baseball diamter in pixel. Then, I use the equation: {**v = dx/dt = abs(x1-x2)/(t1-t2)**} ,to calculate the velocity in z-axis 

In [ ]:
# Initialize the chosen IMG number
chosen = [5,8,10,11,12,13,14,15]

speed_x = [] # speed_x (width direction) 
speed_y = [] # speed_y (height direction)
speed_z = [] # depth speed (normal to the 2D image plane direction)

for i in range(len(chosen)-1):
    circle = circle_list[chosen[i]-1]
    circle_1 = circle_list[chosen[i+1]-1]

    a,b,r =circle[0],circle[1],circle[2] # pixel
    a1,b1,r1 =circle_1[0],circle_1[1],circle_1[2] # pixel

    # Diameter of baseball in pixel
    d = r*2 # pixel
    d1 = r1*2 # pixel

    # Compute x position
    x = a*FoV_W/res_w
    x1 = a1*FoV_W/res_w

    # Compute y position
    y = b*FoV_H/res_h
    y1 = b1*FoV_H/res_h

    # Compute z position
    z = fx*diameter_bb/d
    z1 = fx*diameter_bb/d1 
    
    # Calculate velocity in x,y,z direction
    dx = abs(x-x1)
    dy = abs(y-y1)
    dz = abs(z-z1)
    dt = (chosen[i+1]-chosen[i])/fps
    vel_x = dx/dt
    vel_y = dy/dt
    vel_z = dz/dt

    speed_x.append(vel_x)
    speed_y.append(vel_y)
    speed_z.append(vel_z)
    



In [ ]:
# Calculate average speed (mm/s)
avg_speed_x = sum(speed_x)/len(speed_x)
avg_speed_y = sum(speed_y)/len(speed_y)
avg_speed_z = sum(speed_z)/len(speed_z)

# Convert the speed from mm/s to mph
speed_x_mph = np.around(mmps_to_mph(avg_speed_x),decimals=2)
speed_y_mph = np.around(mmps_to_mph(avg_speed_y),decimals=2)
speed_z_mph = np.around(mmps_to_mph(avg_speed_z),decimals=2)
speed_vector = [speed_x_mph,speed_y_mph,speed_z_mph]

# Calculate speed magnitude
speed_magnitude = np.sqrt(speed_x_mph**2+speed_y_mph**2+speed_z_mph**2)


print('Average speed of baseball = ', [speed_x_mph,speed_y_mph,speed_z_mph])
print('Average speed magnitude of baseball = ', speed_magnitude,'mph')

In [ ]:
# Save IMG with velocity vector except IMG 1 and 2 since the baseball was not being hit off from the bat during that period
for i in range(2,15):
    display_img_vector(img_circle_labeled_list[i],'IMG'+str(i+1),[circle_list[i][0],circle_list[i][1]],speed_vector,i+1)